# Investigation: England "Internal" Transfers 🏴󠁧󠁢󠁥󠁮󠁧󠁿⚽

Same check as Liga MX and Sweden but for England. **Spoiler:** same result — zero internal transfers exist, all records are inter-league (Premier League ↔ Championship).

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 200)

BASE_PATH = "../../thesis_data/"
df_comp = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Wyscout/competitions_wyscout.parquet")
df = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Twelve/male_transfer_model.parquet")
comp_id_to_name = df_comp.drop_duplicates('competition_id').set_index('competition_id')['name'].to_dict()

## 1) English Competitions in Dataset

In [2]:
PREMIER_LEAGUE = 364
CHAMPIONSHIP = 346
ENG_IDS = [PREMIER_LEAGUE, CHAMPIONSHIP]

eng_comps = df_comp[df_comp['country'] == 'England']
eng_comps.groupby(['competition_id', 'name']).agg(
    n_seasons=('season', 'nunique'),
    seasons=('season', lambda x: sorted(x.unique()))
).reset_index()

,competition_id,name,n_seasons,seasons
0,346,Championship,8,"[2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]"
1,364,Premier League,8,"[2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]"


## 2) Zero Internal Transfers

In [3]:
# Global check
same_comp = (df['from_competition'] == df['to_competition']).sum()
print(f"Records where from_competition == to_competition (entire dataset): {same_comp}")
print(f"\n\u274c Zero internal transfers for ANY league, including Premier League.")

Records where from_competition == to_competition (entire dataset): 0

❌ Zero internal transfers for ANY league, including Premier League.


## 3) What ARE the England → England Transfers?

In [4]:
eng = df[
    (df['from_competition'].isin(ENG_IDS)) & (df['to_competition'].isin(ENG_IDS))
].copy()

eng['from_name'] = eng['from_competition'].map(comp_id_to_name)
eng['to_name'] = eng['to_competition'].map(comp_id_to_name)
eng['same_team'] = eng['from_team_id'] == eng['to_team_id']

print(f"Total England\u2192England records: {len(eng)}")
print(f"  Same team (promotion/relegation): {eng['same_team'].sum()}")
print(f"  Different team (player moved clubs between divisions): {(~eng['same_team']).sum()}")

# Breakdown
eng.groupby(['from_competition', 'to_competition', 'from_name', 'to_name']).agg(
    n_records=('player_id', 'count'),
    n_same_team=('same_team', 'sum')
).reset_index()

Total England→England records: 654
  Same team (promotion/relegation): 488
  Different team (player moved clubs between divisions): 166


,from_competition,to_competition,from_name,to_name,n_records,n_same_team
0,346,364,Championship,Premier League,353,239
1,364,346,Premier League,Championship,301,249


In [5]:
# competition_id is stable across seasons
from_364 = df[df['from_competition'] == PREMIER_LEAGUE]
to_364 = df[df['to_competition'] == PREMIER_LEAGUE]

print(f"Premier League (364) FROM seasons: {sorted(from_364['from_season'].unique())}")
print(f"Premier League (364) TO seasons:   {sorted(to_364['to_season'].unique())}")
print(f"\n\u2705 Same ID across all seasons. competition_id = league, not season.")

Premier League (364) FROM seasons: [np.int16(2018), np.int16(2019), np.int16(2020), np.int16(2021), np.int16(2022), np.int16(2023), np.int16(2024)]
Premier League (364) TO seasons:   [np.int16(2018), np.int16(2019), np.int16(2020), np.int16(2021), np.int16(2022), np.int16(2023), np.int16(2024), np.int16(2025)]

✅ Same ID across all seasons. competition_id = league, not season.


---
## 🎯 Conclusion

| Fact | Value |
|------|-------|
| Internal Premier League transfers (364→364) | **0** |
| England→England records | 654 |
| Of which: same team (promo/releg) | 488 |
| Of which: diff team (cross-division) | 166 |
| competition_id stable across seasons? | **Yes** |

All 654 England→England records are **Premier League ↔ Championship** (1st ↔ 2nd division). None are internal transfers within the same league. The dataset by design only contains inter-league transfers.